In [8]:
from netCDF4 import Dataset
import xarray as xr
import dask
import os
import requests
%load_ext autoreload
%autoreload 2

#Allows us to visualize the dask progress for parallel operations
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [11]:
import urllib
from urllib import request
from http.cookiejar import CookieJar
import json
from urllib import request, parse
from http.cookiejar import CookieJar
import getpass
import netrc



def get_token( url: str,client_id: str, user_ip: str,endpoint: str) -> str:
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
        token: str = ''
        xml: str = """<?xml version='1.0' encoding='utf-8'?>
        <token><username>{}</username><password>{}</password><client_id>{}</client_id>
        <user_ip_address>{}</user_ip_address></token>""".format(username, password, client_id, user_ip)
        headers: Dict = {'Content-Type': 'application/xml','Accept': 'application/json'}
        resp = requests.post(url, headers=headers, data=xml)
        response_content: Dict = json.loads(resp.content)
        token = response_content['token']['id']
    except:
        print("Error getting the token - check user name and password")
    return token

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file
    first and if no credentials are found, it prompts for them.
    Valid endpoints include:
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        username = 'soutobias'
        password = '1xW0hTryjnyRACKkO0Aga'

    
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

edl="urs.earthdata.nasa.gov"
cmr="cmr.earthdata.nasa.gov"

setup_earthdata_login_auth(edl)
token_url="https://"+cmr+"/legacy-services/rest/tokens"
token=get_token(token_url,'podaac-subscriber', "127.0.0.1",edl)


Please provide your Earthdata Login credentials to allow data access
Your credentials will only be passed to urs.earthdata.nasa.gov and will not be exposed in Jupyter
Error getting the token - check user name and password


UnboundLocalError: local variable 'token' referenced before assignment